In [1]:
import gzip
import io
import numpy as np
from astropy.io import fits

In [2]:
hdul = fits.HDUList(
    [fits.PrimaryHDU()]
    + [
        fits.ImageHDU(
            data=np.random.randn(1000, 1000),
            header=fits.Header({f"KEY{i}{k}": k for k in range(1000)}),
            name=f"IMAGE{i}",
        )
        for i in range(20)
    ]
)
hdul.writeto("~/test.fits", overwrite=True)
hdul.writeto("~/test.fits.gz", overwrite=True)

In [3]:
ls -lh /home/simon/test.fits*

-rw-r--r-- 1 simon simon 155M May 18 15:21 /home/simon/test.fits
-rw-r--r-- 1 simon simon 147M May 18 15:21 /home/simon/test.fits.gz


In [4]:
%timeit fits.getdata('test.fits', ext=-1)

12.8 ms ± 125 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
%timeit fits.getdata('test.fits.gz', ext=-1)

1.36 s ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit gzip.open('test.fits.gz').read()

702 ms ± 4.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit fits.getdata(io.BytesIO(gzip.open('test.fits.gz').read()), ext=-1)

714 ms ± 4.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
